In [1]:
%matplotlib inline
from solvers import solver
import numpy as np
import qtools as qt
import tucker3d as tuck

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 29 days
openbabel not found in path, switching to PyQuante backend
libint extension not found, switching to normal ERI computation


In [2]:
name = 'He'
molecule = qt.molecule(name)


sol = solver(molecule, method='hf', mixing=5, eps=1e-5,
            maxiter=60, meshsize=1024, boxsize=10.)
sol.solve()
print sol.energy

SCF iteration - done
0.00812101364136
(3, 3, 3)
Iteration 0 accuracy = 2.70e-02
Iteration 1 accuracy = 2.35e-02
Iteration 2 accuracy = 2.31e-03
Iteration 3 accuracy = 3.56e-03
Iteration 4 accuracy = 2.13e-03
Iteration 5 accuracy = 4.21e-04
Iteration 6 accuracy = 1.30e-04
Iteration 7 accuracy = 1.08e-04
Iteration 8 accuracy = 2.48e-05
Iteration 9 accuracy = 7.65e-06
Iteration 10 accuracy = 7.68e-06
Iteration 11 accuracy = 1.71e-06
Iteration 12 accuracy = 1.85e-06
Process converged with 12 iterations
Full energy =  -2.86057539624
-2.86057539624


solvers/pyquante.py:47: ComplexWarning: Casting complex values to real discards the imaginary part
  psi[i] = gto2tuck(w, cf[:,i], x, eps)
